In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

In [2]:
df = pd.read_csv("/Users/mamatharaj/Desktop/python/aws_review_sofware_dataset.csv")

/var/folders/_8/gygsnz5n09n9kvs6h09qf02m0000gn/T/ipykernel_90304/3082548048.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mamatharaj/Desktop/python/aws_review_sofware_dataset.csv")


In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [4]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

In [5]:
from nltk.tokenize import sent_tokenize

# Generating features from the review text data column

In [6]:
for i in range(499):
    l1= sent_tokenize(df.loc[i,"reviewText"])
    df.at[i,"sentences"]=l1

# Generating features using Sentence Lemmatization

In [7]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 4.492929220199585 secs.


In [8]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [9]:
df.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,words,sentences
0,0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN,"[the, material, arrive, early, and, be, in, ex...",[The materials arrived early and were in excel...
1,1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN,"[i, be, really, enjoy, this, book, with, the, ...",[I am really enjoying this book with the works...
2,2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7,NaN,"[if, you, be, take, this, class, don, '', t, w...","[IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR..."
3,3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3,NaN,"[this, book, be, miss, page, !, !, !, importan...","[This book was missing pages!!!, Important pag..."
4,4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN,"[i, have, use, learnsmart, and, can, officiall...",[I have used LearnSmart and can officially say...


In [10]:
df["words_sentences"] = [' ' for i in range(459436)]

In [11]:
def get_str(list1):
    s1 = ''
    for i1 in list1:
        s1 = s1+' '+i1
    return s1

In [12]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=get_str(df.loc[k,"words"])

# Using Tfidf Vectorization

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

In [15]:
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [16]:
dense.head()

,00,000,0000,0321670094,0321701801,0321719808,0789744449,0824831659,10,100,...,years,yellow,yes,yesterday,yo,young,youtube,yr,zero,zoom
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.324845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_y=df["verified"]

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le=LabelEncoder()

In [20]:
df_y_1=pd.DataFrame(df_y)

In [21]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [22]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


# Applying Classfication algorithms on generated features

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

In [24]:
GBC=GradientBoostingClassifier(n_estimators=3000)

In [ ]:
gb_c=GBC.fit(dense,df_y_enc)

/Users/mamatharaj/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
GBC.score(dense,df_y_enc)

In [23]:
from sklearn.tree import DecisionTreeClassifier

In [24]:
DecTree=DecisionTreeClassifier()

In [25]:
dt_M=DecTree.fit(dense,df_y_enc)

In [26]:
dt_M.score(dense,df_y_enc)

0.6735976283965558